### Chapter 11 - Regex

Exercise 11.1: 
    
Write a program to look for lines of the form:
    
New Revision: 39772
    
Extract the number from each of the lines using a regular expression and the findall() method. Compute the average of the numbers and print out the average.

In [ ]:
import re

tot = 0
count = 0
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
  
    x = re.findall('^New .*: ([0-9]+)', line)
    for i in x:
        if len(x) > 0:
            tot += float(i)
            count += 1
            
print(tot/count)

### Chapter 12 - Networked Programs

Exercise 12.1:
    
Change the socket program socket1.py to prompt the user for the URL so it can read any web page. You can use split('/') to break the URL into its component parts so you can extract the host name for the socket connect call. Add error checking using try and except to handle the condition where the user enters an improperly formatted or non-existent URL.

In [ ]:
import socket
mysock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    mysock.connect(('data.pr4e.org', 90))
    cmd = 'GET http://data.pr4e.org/romeo.txt HTTP/1.0\r\n\r\n'.encode()
    mysock.send(cmd)
except socket.error as msg:
    print("BOBO")
    mysock.close()
while True:
    data = mysock.recv(512)
    if len(data) < 1:
        break
    print(data.decode(),end='')
mysock.close()

Exercise 12.2:

Change your socket program so that it counts the number of characters it has received and stops displaying any text after it has shown 3000 characters. The program should retrieve the entire document and count the total number of characters and display the count of the number of characters at the end of the document

In [ ]:
import socket
import time
HOST = 'data.pr4e.org'
PORT = 80
mysock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
mysock.connect((HOST, PORT))
mysock.sendall(b'GET http://data.pr4e.org/cover3.jpg HTTP/1.0\r\n\r\n')
count = 0
while True:
    data = mysock.recv(512)
    if len(data) < 1:break
    count +=len(data)
    if count > 3000:break
    print(len(data), count)
mysock.close()


Exercise 12.3:

Use urllib to replicate the previous exercise of (1) retrieving the document from a URL, (2) displaying up to 3000 characters, and (3) counting the overall number of characters in the document. Don't worry about the headers for this exercise, simply show the first 3000 characters of the document contents.

In [ ]:
import urllib.request
fhand = urllib.request.urlopen('http://data.pr4e.org/romeo.txt')
count = 0
for line in fhand:
    words = line.decode().strip()
    count += len(words)
    print(words[:3001])
print(count)

Exercise 12.4:
    
(Advanced) Change the socket program so that it only shows data after the headers and a blank line have been received. Remember that recv receives characters (newlines and all), not lines.

In [ ]:
import socket
import re

mysock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
mysock.connect(('data.pr4e.org', 80))
cmd = 'GET http://data.pr4e.org/romeo.txt HTTP/1.0\n\n'.encode()
mysock.send(cmd)

while True:                                 
    data = mysock.recv(512)
    if len(data) < 1: break
    message = data.decode()
    pos = data.find(b"\r\n\r\n")
    message2 = message[pos+4:]
    print(message2)
mysock.close()

### Chapter 13 - Python & Web Services

Exercise: Change either the www.py4e.com/code3/geojson.py or www.py4e.com/code3/geoxml.py
to print out the two-character country code from the retrieved data. Add error
checking so your program does not traceback if the country code is not there.
Once you have it working, search for “Atlantic Ocean” and make sure it can
handle locations that are not in any country.

In [ ]:
import urllib.request, urllib.parse, urllib.error
import json

# Note that Google is increasingly requiring keys
# for this API
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/json?'

while True:
    address = input('Enter location: ')
    if len(address) < 1: break

    url = serviceurl + urllib.parse.urlencode(
        {'address': address})

    print('Retrieving', url)
    uh = urllib.request.urlopen(url)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters')

    try:
        js = json.loads(data)
    except:
        js = None

    if not js or 'status' not in js or js['status'] != 'OK':
        print('==== Failure To Retrieve ====')
        print(data)
        continue

#    print(json.dumps(js, indent=4))
    counter = -1
    info = js["results"][0]["address_components"]
    for item in info:
        counter += 1
        if js["results"][0]["address_components"][counter]["types"] == ['country', 'political']:
            print(js["results"][0]["address_components"][counter]["short_name"])
        else:
            continue
        
